In [ ]:
# ----------------------------------------------------------------------
# Optuna (베이지안 최적화 기반) 정의 및 원리
# ----------------------------------------------------------------------

# 1. 정의:
# - 이전 시도의 평가 결과를 바탕으로 다음 시도에 가장 성능이 좋을 것으로 '예측'되는 조합을 '지능적으로' 선택하여 탐색하는 베이지안 최적화 기반 프레임워크.
# - Random Search보다 훨씬 적은 반복만으로 최적의 하이퍼파라미터를 찾을 수 있어 가장 효율적임.

# 2. 핵심 원리:
# - Define-by-Run: 탐색 공간을 미리 고정하지 않고, 튜닝 과정(Run-time) 중에 동적으로(조건에 따라) 정의할 수 있음.
# - 베이지안 최적화: 확률 모델(대리 모델)을 사용하여 탐색할 다음 위치를 결정.
#   - 탐험 (Exploration): 아직 시도해보지 않은 새로운 영역을 탐색.
#   - 활용 (Exploitation): 이전에 성능이 좋았던 영역 주변을 집중적으로 탐색.
#   - Optuna는 이 두 가지를 균형 있게 수행하여 효율을 극대화함.

# 3. 주요 요소:
# - Study (스터디): 전체 최적화 세션을 관리하는 객체. 최적화 방향(maximize/minimize)을 정의함.
# - Trial (시행): 한 번의 하이퍼파라미터 조합을 시도하고 평가하는 과정.
# - Objective Function (목적 함수): Optuna로부터 제안받은 조합으로 모델을 학습시키고, 최적화할 점수(Scalar Value)를 반환하는 함수. (Optuna가 이 값을 최대화 또는 최소화하려고 시도함)

# 4. 장점:
# - 높은 효율성: 비효율적인 영역 탐색을 피하여 시간과 자원을 크게 절약.
# - 유연성: 조건부 파라미터나 복잡한 탐색 공간을 동적으로 처리 가능.
# - 강력한 시각화: 탐색 과정을 시각적으로 분석할 수 있는 다양한 도구 제공.

In [1]:
# --------------------
#  라이브러리 및 데이터 준비
# --------------------
# pip install optuna scikit-learn
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score, train_test_split

# Iris 데이터셋 로드
iris = load_iris()
X, y = iris.data, iris.target

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("--- 데이터 준비 완료 ---")

--- 데이터 준비 완료 ---


In [2]:
# --------------------
# 2단계: 목적 함수 (Objective Function) 정의
# --------------------
def objective(trial):
    """
    Optuna로부터 하이퍼파라미터를 제안받아 모델을 평가하고 점수를 반환하는 함수.
    """
    
    # 1. 하이퍼파라미터 탐색 공간 정의 (trial.suggest_*)
    # Optuna는 이전에 평가된 결과를 바탕으로 다음에 시도할 최적의 값을 지능적으로 제안합니다.
    n_estimators = trial.suggest_int('n_estimators', 100, 500)  # 정수형 범위 탐색
    max_depth = trial.suggest_int('max_depth', 3, 10)           # 정수형 범위 탐색
    min_samples_split = trial.suggest_float('min_samples_split', 0.01, 0.1) # 실수형 범위 탐색
    
    # 2. 모델 생성 및 교차 검증 평가
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        random_state=42
    )
    
    # 5-Fold 교차 검증을 사용하여 정확도(Accuracy)를 평가
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    
    # 교차 검증 점수의 평균을 반환 (Optuna는 이 값을 최대화하려고 시도함)
    return score.mean()

print("\n--- 2단계: 목적 함수 정의 완료 ---")


--- 2단계: 목적 함수 정의 완료 ---


In [3]:
# --------------------
# 3단계: Optuna Study 실행 및 최적화
# --------------------
# direction="maximize": 목적 함수의 반환값(정확도)을 최대화하도록 설정
study = optuna.create_study(direction="maximize") 

# optimize 실행: 목적 함수를 호출하여 50번의 반복(trial) 동안 최적의 조합을 탐색
# n_trials=50: 총 50가지 조합만 시도 (Grid Search 대비 매우 효율적)
study.optimize(objective, n_trials=50, show_progress_bar=True) 

[I 2025-10-17 01:52:30,428] A new study created in memory with name: no-name-ad2db1f2-cae2-4d3d-bb08-22db837ca8bb


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-17 01:52:33,705] Trial 0 finished with value: 0.9428571428571428 and parameters: {'n_estimators': 190, 'max_depth': 3, 'min_samples_split': 0.08690077875407516}. Best is trial 0 with value: 0.9428571428571428.
[I 2025-10-17 01:52:38,964] Trial 1 finished with value: 0.9428571428571428 and parameters: {'n_estimators': 319, 'max_depth': 8, 'min_samples_split': 0.012968108543957113}. Best is trial 0 with value: 0.9428571428571428.
[I 2025-10-17 01:52:42,234] Trial 2 finished with value: 0.9428571428571428 and parameters: {'n_estimators': 135, 'max_depth': 7, 'min_samples_split': 0.09884559905823312}. Best is trial 0 with value: 0.9428571428571428.
[I 2025-10-17 01:52:50,177] Trial 3 finished with value: 0.9428571428571428 and parameters: {'n_estimators': 358, 'max_depth': 8, 'min_samples_split': 0.07711712947697456}. Best is trial 0 with value: 0.9428571428571428.
[I 2025-10-17 01:53:00,914] Trial 4 finished with value: 0.9428571428571428 and parameters: {'n_estimators': 420, '

In [4]:
# --------------------
# 4단계: 결과 확인
# --------------------
print("\n--- 4단계: 최적화 결과 확인 ---")
# 최적의 하이퍼파라미터 조합 (Best Parameters)
print("Best Parameters:", study.best_params)

# 해당 조합으로 얻은 최고 점수 (Best Value)
print(f"Best CV Score (Accuracy): {study.best_value:.4f}")

# 최종 모델 학습 (최적 파라미터를 사용하여 최종 모델 재학습)
best_params = study.best_params
final_model = RandomForestClassifier(**best_params, random_state=42)
final_model.fit(X_train, y_train)

final_test_score = final_model.score(X_test, y_test)
print(f"Final Test Score: {final_test_score:.4f}")


--- 4단계: 최적화 결과 확인 ---
Best Parameters: {'n_estimators': 190, 'max_depth': 3, 'min_samples_split': 0.08690077875407516}
Best CV Score (Accuracy): 0.9429
Final Test Score: 1.0000
